# Distributed Data Parallel in Pytorch

`torch.nn.parallel.DistributedDataParallel`相比于 `torch.nn.DataParallel`效率更高,由多线程执行, 不受GIL锁限制

In [15]:
import torch
import torch.distributed
import argparse

In [16]:
parser = argparse.ArgumentParser()
parser.add_argument('--local_rank', type=int, default=5)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--local_rank LOCAL_RANK]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/panjiahua/.local/share/jupyter/runtime/kernel-be274211-e00b-4f93-afdc-0145dd6f6da3.json


SystemExit: 2

流程
1. 初始化一个进程组

In [6]:
torch.distributed.is_available()

True

-- backend: GPU之间的通讯协议, 一般使用`nccl`

-- world_size: 参与工作的进程数, 一般来说单机的话填入有多少张GPU 这里假设为"N"张

-- rank: args.local_rank  -- 指定当前进程在哪个GPU卡上

In [ ]:
torch.distributed.init_process_group(
    backend='nccl', world_size=N, rank=args.local_rank
)

-- 2.指定当前这个进程可以用到的GPU的名称

In [1]:
torch.cuda.set_device(args.local_rank)

## 相当于是用了CUDA_VISIBLE_DEVICES环境变量

NameError: name 'torch' is not defined

-- 3.对模型进行打包

In [ ]:
## torch.nn.parallel.DistributedDataParallel 和 torch.utils.data.distributed.DistributedSampler要配合使用

model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[local_rank])

-- 4.准备Sampler, Pytorch提供了`torch.utils.data.DistributedSampler`来为各个进程切分数据，以保证训练数据不重叠

In [ ]:
train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)

# 在每个周期开始处，调用train_sampler.set_epoch(epoch)可以使得数据充分打乱
# for epoch in range(start_epoch, n_epochs):
#         ...     if is_distributed:
#         ...         sampler.set_epoch(epoch)

In [ ]:
## Dataloader 里的shuffle设置为False, Sampler设置为定义好的分布式的sampler
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True, sampler=train_sampler)

-- 5. 执行命令

`python -m torch.distributed.launch --nproc_per_node=n_gpus
           train.py (--arg1 --arg2 --arg3 and all other
           arguments of your training script)`

In [2]:
# 模型保存
# 在local_rank == 0时才保存



In [ ]:
from torch.utils.data import distributed